In [1]:
import pandas as pd
import afqinsight.nn.tf_models as nn
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from afqinsight.datasets import AFQDataset
from afqinsight.nn.tf_models import cnn_lenet, mlp4, cnn_vgg, lstm1v0, lstm1, lstm2, blstm1, blstm2, lstm_fcn, cnn_resnet
from sklearn.impute import SimpleImputer
import os.path
# Harmonization
from sklearn.model_selection import train_test_split
from neurocombat_sklearn import CombatModel
from sklearn.metrics import r2_score, median_absolute_error, mean_absolute_error

import pickle
from tools import load_data, model_fit

In [2]:
X, y, site = load_data()

2022-07-07 02:24:04.506119: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
from sklearn.utils import shuffle, resample

In [4]:
from sklearn.metrics import r2_score, median_absolute_error

In [5]:
model_dict = {
#   "cnn_lenet": {"model": cnn_lenet, "lr": 0.001}, 
  "mlp4": {"model": mlp4, "lr": 0.001},
  # "cnn_vgg": {"model": cnn_vgg, "lr": 0.001},
  # "lstm1v0": {"model": lstm1v0, "lr": 0.01},
  # "lstm1": {"model": lstm1, "lr": 0.01},
  # "lstm2": {"model": lstm2, "lr": 0.01},
  # "blstm1": {"model": blstm1, "lr": 0.01},
  # "blstm2": {"model": blstm1, "lr": 0.01},
  # "lstm_fcn": {"model": lstm_fcn, "lr": 0.01},
#   "cnn_resnet": {"model": cnn_resnet, "lr": 0.01}
             }

In [6]:
n_runs = 10

In [7]:
# Generated once with this code and then hard-coded:
# seeds = np.array([np.abs(np.floor(np.random.randn()*1000)) for ii in range(n_runs)], dtype=int)

seeds = np.array([484, 645, 714, 244, 215, 1502, 1334, 1576, 469, 1795])

In [8]:
def fit_and_eval(model, X, y, random_state, batch_size=128, n_epochs=1000, augment=None, train_size=None):
    model_func = model_dict[model]["model"]
    lr = model_dict[model]["lr"]
    X_train, X_test, y_train, y_test, site_train, site_test = train_test_split(X, y, site, test_size=0.2, random_state=random_state)
    imputer = SimpleImputer(strategy="median")
    # If train_size is set, select train_size subjects to be the training data:
    if train_size is not None:
        X_train, y_train, site_train = resample(X_train, y_train, site_train, replace=False, n_samples=train_size, random_state=random_state)
    
    # Impute train and test separately:
    X_train = np.concatenate([imputer.fit_transform(X_train[..., ii])[:, :, None] for ii in range(X_train.shape[-1])], -1)
    X_test = np.concatenate([imputer.fit_transform(X_test[..., ii])[:, :, None] for ii in range(X_test.shape[-1])], -1)
    # Combat
    X_train = np.concatenate([CombatModel().fit_transform(X_train[..., ii], site_train[:, None], None, None)[:, :, None] for ii in range(X_train.shape[-1])], -1)
    X_test = np.concatenate([CombatModel().fit_transform(X_test[..., ii], site_test[:, None], None, None)[:, :, None] for ii in range(X_test.shape[-1])], -1)
    
    trained = model_fit(model_func, X_train, y_train, lr, 
                        batch_size=batch_size, n_epochs=n_epochs)
    metric = []
    value = []
    
    y_pred = trained.predict(X_test)
    metric.append("mae")
    value.append(mean_absolute_error(y_test, y_pred))
    metric.append("mad")
    value.append(median_absolute_error(y_test, y_pred))
    metric.append("r2")
    value.append(r2_score(y_test, y_pred))
    
    result = {'Model': [model] * len(metric),
              'Metric': metric,
              'Value': value}
    
    return pd.DataFrame(result), pd.DataFrame(dict(y_pred=y_pred.squeeze(), y_test=y_test))

In [9]:
dfs_eval = []
dfs_pred = []

In [10]:
for model in model_dict:
    print("##################################################")
    print("model: ", model)
    for ii in range(n_runs): 
        print("run: ", ii)
        this_eval, this_pred = fit_and_eval(
            model, 
            X, 
            y, 
            random_state=seeds[ii],
            train_size=100)
        this_eval["run"] = ii
        this_pred["run"] = ii
        dfs_eval.append(this_eval)
        dfs_pred.append(this_pred)
        # Save evaluation metrics
        one_df = pd.concat(dfs_eval)
        one_df.to_csv("mlp4_6_eval.csv")
        # Save predictions and test values:
        one_df = pd.concat(dfs_pred)
        one_df.to_csv("mlp4_6_pred.csv")

##################################################
model:  mlp4
run:  0
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 100, 72)]         0         
                                                                 
 flatten (Flatten)           (None, 7200)              0         
                                                                 
 dropout (Dropout)           (None, 7200)              0         
                                                                 
 dense (Dense)               (None, 500)               3600500   
                                                                 
 dropout_1 (Dropout)         (None, 500)               0         
                                                                 
 dense_1 (Dense)             (None, 500)               250500    
                                                       

1/1 [==============================] - 0s 74ms/step - loss: 12.7626 - mean_squared_error: 12.7626 - rmse: 3.5725 - mean_absolute_error: 2.9787 - val_loss: 11.7994 - val_mean_squared_error: 11.7994 - val_rmse: 3.4350 - val_mean_absolute_error: 2.7408 - lr: 0.0010
Epoch 25/1000
1/1 [==============================] - 0s 65ms/step - loss: 13.2165 - mean_squared_error: 13.2165 - rmse: 3.6354 - mean_absolute_error: 2.9366 - val_loss: 12.1588 - val_mean_squared_error: 12.1588 - val_rmse: 3.4869 - val_mean_absolute_error: 2.6775 - lr: 0.0010
Epoch 26/1000
1/1 [==============================] - 0s 79ms/step - loss: 11.0952 - mean_squared_error: 11.0952 - rmse: 3.3310 - mean_absolute_error: 2.6867 - val_loss: 13.0694 - val_mean_squared_error: 13.0694 - val_rmse: 3.6152 - val_mean_absolute_error: 2.6977 - lr: 0.0010
Epoch 27/1000
1/1 [==============================] - 0s 77ms/step - loss: 10.1443 - mean_squared_error: 10.1443 - rmse: 3.1850 - mean_absolute_error: 2.6647 - val_loss: 15.2085 - val_

Epoch 52/1000
1/1 [==============================] - 0s 68ms/step - loss: 10.3755 - mean_squared_error: 10.3755 - rmse: 3.2211 - mean_absolute_error: 2.6505 - val_loss: 15.0535 - val_mean_squared_error: 15.0535 - val_rmse: 3.8799 - val_mean_absolute_error: 2.8199 - lr: 2.5000e-04
Epoch 53/1000
1/1 [==============================] - 0s 49ms/step - loss: 11.3803 - mean_squared_error: 11.3803 - rmse: 3.3735 - mean_absolute_error: 2.7907 - val_loss: 15.7678 - val_mean_squared_error: 15.7678 - val_rmse: 3.9709 - val_mean_absolute_error: 2.8607 - lr: 2.5000e-04
Epoch 54/1000
1/1 [==============================] - 0s 64ms/step - loss: 10.8844 - mean_squared_error: 10.8844 - rmse: 3.2992 - mean_absolute_error: 2.6821 - val_loss: 16.5923 - val_mean_squared_error: 16.5923 - val_rmse: 4.0734 - val_mean_absolute_error: 2.9211 - lr: 2.5000e-04
Epoch 55/1000
1/1 [==============================] - 0s 53ms/step - loss: 9.7909 - mean_squared_error: 9.7909 - rmse: 3.1290 - mean_absolute_error: 2.5908 - 

1/1 [==============================] - 0s 98ms/step - loss: 10.8471 - mean_squared_error: 10.8471 - rmse: 3.2935 - mean_absolute_error: 2.6164 - val_loss: 16.3715 - val_mean_squared_error: 16.3715 - val_rmse: 4.0462 - val_mean_absolute_error: 2.9038 - lr: 1.2500e-04
Epoch 81/1000
1/1 [==============================] - 0s 64ms/step - loss: 9.9365 - mean_squared_error: 9.9365 - rmse: 3.1522 - mean_absolute_error: 2.5972 - val_loss: 16.2440 - val_mean_squared_error: 16.2440 - val_rmse: 4.0304 - val_mean_absolute_error: 2.8948 - lr: 1.2500e-04
Epoch 82/1000
1/1 [==============================] - 0s 103ms/step - loss: 8.9780 - mean_squared_error: 8.9780 - rmse: 2.9963 - mean_absolute_error: 2.3242 - val_loss: 16.0885 - val_mean_squared_error: 16.0885 - val_rmse: 4.0110 - val_mean_absolute_error: 2.8834 - lr: 1.2500e-04
Epoch 83/1000
1/1 [==============================] - 0s 64ms/step - loss: 10.1327 - mean_squared_error: 10.1327 - rmse: 3.1832 - mean_absolute_error: 2.5512 - val_loss: 15.94

1/1 [==============================] - ETA: 0s - loss: 11.3923 - mean_squared_error: 11.3923 - rmse: 3.3752 - mean_absolute_error: 2.7625
Epoch 00108: ReduceLROnPlateau reducing learning rate to 3.125000148429535e-05.
1/1 [==============================] - 0s 79ms/step - loss: 11.3923 - mean_squared_error: 11.3923 - rmse: 3.3752 - mean_absolute_error: 2.7625 - val_loss: 17.2488 - val_mean_squared_error: 17.2488 - val_rmse: 4.1532 - val_mean_absolute_error: 2.9643 - lr: 6.2500e-05
run:  1
Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 100, 72)]         0         
                                                                 
 flatten_1 (Flatten)         (None, 7200)              0         
                                                                 
 dropout_4 (Dropout)         (None, 7200)              0         
                            

...

Model: "model_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_6 (InputLayer)        [(None, 100, 72)]         0         
                                                                 
 flatten_5 (Flatten)         (None, 7200)              0         
                                                                 
 dropout_20 (Dropout)        (None, 7200)              0         
                                                                 
 dense_20 (Dense)            (None, 500)               3600500   
                                                                 
 dropout_21 (Dropout)        (None, 500)               0         
                                                                 
 dense_21 (Dense)            (None, 500)               250500    
                                                                 
 dropout_22 (Dropout)        (None, 500)               0   

1/1 [==============================] - 0s 68ms/step - loss: nan - mean_squared_error: nan - rmse: nan - mean_absolute_error: nan - val_loss: nan - val_mean_squared_error: nan - val_rmse: nan - val_mean_absolute_error: nan - lr: 5.0000e-04
Epoch 27/1000
1/1 [==============================] - 0s 80ms/step - loss: nan - mean_squared_error: nan - rmse: nan - mean_absolute_error: nan - val_loss: nan - val_mean_squared_error: nan - val_rmse: nan - val_mean_absolute_error: nan - lr: 5.0000e-04
Epoch 28/1000
1/1 [==============================] - 0s 55ms/step - loss: nan - mean_squared_error: nan - rmse: nan - mean_absolute_error: nan - val_loss: nan - val_mean_squared_error: nan - val_rmse: nan - val_mean_absolute_error: nan - lr: 5.0000e-04
Epoch 29/1000
1/1 [==============================] - 0s 66ms/step - loss: nan - mean_squared_error: nan - rmse: nan - mean_absolute_error: nan - val_loss: nan - val_mean_squared_error: nan - val_rmse: nan - val_mean_absolute_error: nan - lr: 5.0000e-04
Ep

1/1 [==============================] - 0s 59ms/step - loss: nan - mean_squared_error: nan - rmse: nan - mean_absolute_error: nan - val_loss: nan - val_mean_squared_error: nan - val_rmse: nan - val_mean_absolute_error: nan - lr: 2.5000e-04
Epoch 58/1000
1/1 [==============================] - 0s 57ms/step - loss: nan - mean_squared_error: nan - rmse: nan - mean_absolute_error: nan - val_loss: nan - val_mean_squared_error: nan - val_rmse: nan - val_mean_absolute_error: nan - lr: 2.5000e-04
Epoch 59/1000
1/1 [==============================] - 0s 93ms/step - loss: nan - mean_squared_error: nan - rmse: nan - mean_absolute_error: nan - val_loss: nan - val_mean_squared_error: nan - val_rmse: nan - val_mean_absolute_error: nan - lr: 2.5000e-04
Epoch 60/1000
1/1 [==============================] - ETA: 0s - loss: nan - mean_squared_error: nan - rmse: nan - mean_absolute_error: nan
Epoch 00060: ReduceLROnPlateau reducing learning rate to 0.0001250000059371814.
1/1 [==============================] 

1/1 [==============================] - 0s 69ms/step - loss: nan - mean_squared_error: nan - rmse: nan - mean_absolute_error: nan - val_loss: nan - val_mean_squared_error: nan - val_rmse: nan - val_mean_absolute_error: nan - lr: 6.2500e-05
Epoch 88/1000
1/1 [==============================] - 0s 74ms/step - loss: nan - mean_squared_error: nan - rmse: nan - mean_absolute_error: nan - val_loss: nan - val_mean_squared_error: nan - val_rmse: nan - val_mean_absolute_error: nan - lr: 6.2500e-05
Epoch 89/1000
1/1 [==============================] - 0s 69ms/step - loss: nan - mean_squared_error: nan - rmse: nan - mean_absolute_error: nan - val_loss: nan - val_mean_squared_error: nan - val_rmse: nan - val_mean_absolute_error: nan - lr: 6.2500e-05
Epoch 90/1000
1/1 [==============================] - 0s 131ms/step - loss: nan - mean_squared_error: nan - rmse: nan - mean_absolute_error: nan - val_loss: nan - val_mean_squared_error: nan - val_rmse: nan - val_mean_absolute_error: nan - lr: 6.2500e-05
E

FileNotFoundError: [Errno 2] Unable to open file (unable to open file: name = '/var/folders/c_/8lvtjqcn13jcbbdq_sx7f7sr0000gn/T/tmppoe7_7do.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

In [ ]:
# We want to try different training sample sizes: s
# (1453, 1000, 700, 350, 175)